<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Investigaci-n-de-operaciones/blob/main/TareaIO4/Modelo_de_transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import numpy as np

In [72]:
class ModeloTransporte:
  def asignar(Ma,oferta,demanda, i,j):
    if oferta[i]==0 and demanda[i]==0:
      return None
    #Se encarga de asignar la oferta, revisa si se puede colocar la oferta disponible o si no coloca la demanda disponible
    if oferta[i]<=demanda[j]: 
      Ma[i][j]=oferta[i]
    else:
    #Elimina la demanda y oferta de sus vectores
      Ma[i][j]=demanda[j]
    oferta[i]-=Ma[i][j]
    demanda[j]-=Ma[i][j]

  def eliminar(K,oferta,demanda):
    for i in range(len(oferta)):
      if(oferta[i]==0):
        K[i,:]=np.ones(len(K[i,:]))*1e5
    for i in range(len(demanda)):
      if(demanda[i]==0):
        K[:,i]=np.ones(len(K[:,i]))*1e5

  def calcular(C,M):#La suma de la diagonal de la matriz de resultados y la matriz de costos es igual a calcular el costo
    S=np.array(C).T@np.array(M)
    sol=sum([S[i][i] for i in range(len(S))])
    return sol


  def solNO(Ma,oferta,demanda,i=0,j=0):
    #Solucion noroeste
    #Asigna un valor a la posicion i j
    ModeloTransporte.asignar(Ma,oferta,demanda,i,j)

    #si la oferta o la demanda no es igual a cero despues de la asignacion asigna a la siguiente casilla
    if oferta[i]!=0:
      return ModeloTransporte.solNO(Ma,oferta,demanda,i,j+1)
    elif demanda[j]!=0:
      return ModeloTransporte.solNO(Ma,oferta,demanda,i+1,j)
    #Si la oferta y la demanda son cero y ademas la suma de vectores demanda y oferta es igual a cero, entonces se terminó el ejercicio, si no pasa la la submatriz del siguiente paso
    if oferta[i]==0 and demanda[j]==0 and sum(demanda)+sum(oferta)!=0:
      return ModeloTransporte.solNO(Ma,oferta,demanda,i+1,j+1)
    
    return Ma,oferta,demanda


  def norOeste(C,oferta,demanda):
    #Construye la matriz de resultados
    M=[[0 for i in C[0]] for i in C]
    ModeloTransporte.solNO(M,oferta,demanda)
    return M,ModeloTransporte.calcular(C,M)
    
  def minimoCosto(C,oferta,demanda,M=None,K=None):
    #Hace una copia de la matriz de costos
    if M==None:
      K=np.array(C)
      M=[[0 for i in C[0]] for i in K]
    #Recorre las columnas de la matriz, selecciona el minimo y le asigna un valor
    for i in range(len(C[0])):
      ModeloTransporte.asignar(M,oferta,demanda,np.argmin(K.T[i]),i)
    
    ModeloTransporte.eliminar(K,oferta,demanda)
    
    if sum(demanda)+sum(oferta)!=0:
      ModeloTransporte.minimoCosto(C,oferta,demanda,M,K)
    
    return M,ModeloTransporte.calcular(C,M)
    

  def vogel(C,oferta,demanda):
    M=[[0 for i in C[0]] for i in C]
    K=np.array(C)
    pf=[abs(np.sort(K[:,i])[0]-np.sort(K[:,i])[1]) for i in range(len(demanda))]
    pc=[abs(np.sort(K[i,:])[0]-np.sort(K[i,:])[1]) for i in range(len(oferta))]
    
    if np.max(pf)>np.max(pc):
      x,y=np.argmin(K[:,np.argmax(pf)]),np.argmax(pf)
    elif np.max(pf)<np.max(pc):
      x,y=np.argmax(pc),np.argmin(K[np.argmax(pc),:])
    #else:
      #ModeloTransporte.asignar(M,oferta,demanda,np.argmin(K[:,np.argmax(pf)]),np.argmax(pf))
    
    ModeloTransporte.asignar(M,oferta,demanda,x,y)
    ModeloTransporte.eliminar(K,oferta,demanda)
    while demanda[x]!=0:
      x,y=np.argmax(pc),np.argmin(K[np.argmax(pc),:])
      ModeloTransporte.asignar(M,oferta,demanda,x,y)
      ModeloTransporte.eliminar(K,oferta,demanda)
    while oferta[y]!=0:
      x,y=np.argmin(K[:,np.argmax(pf)]),np.argmax(pf)
      ModeloTransporte.asignar(M,oferta,demanda,x,y)
      ModeloTransporte.eliminar(K,oferta,demanda)
    print(pf)
    print(pc)
    print(M)
  

In [69]:
C=[[10,2,20,11],[12,7,9,20],[4,14,16,18]]
oferta=[15,25,10]
demanda=[5,15,15,15]
ModeloTransporte.norOeste(C,oferta,demanda)

([[5, 10, 0, 0], [0, 5, 15, 5], [0, 0, 0, 10]], 520)

In [55]:
C=[[10,2,20,11],[12,7,9,20],[4,14,16,18]]
oferta=[15,25,10]
demanda=[5,15,15,15]
ModeloTransporte.minimoCosto(C,oferta,demanda)

([[0, 15, 0, 0], [0, 0, 15, 10], [5, 0, 0, 5]], 475)

In [73]:
C=[[10,2,20,11],[12,7,9,20],[4,14,16,18]]
oferta=[15,25,10]
demanda=[5,15,15,15]
ModeloTransporte.vogel(C,oferta,demanda)

[6, 5, 7, 7]
[8, 2, 10]
[[0, 0, 0, 0], [0, 0, 0, 0], [5, 0, 0, 0]]
